This widget tests the following
* Sending messages from kernel to js & vice versa
* Sending binary messages from kernel to js & vice versa

In [ ]:
import anywidget
import traitlets

class ByteMemoryView(traitlets.TraitType):
    """A trait for memory views of bytes."""

    default_value = memoryview(b'')
    info_text = 'a memory view object'

    def validate(self, obj, value):
        if isinstance(value, memoryview) and value.format == 'B':
            return value
        self.error(obj, value)

    def default_value_repr(self):
        return repr(self.default_value.tobytes())


class SampleWidget(anywidget.AnyWidget):
    # Widget front-end JavaScript code
    _esm = """
    export function render({ model, el }) {
      let button = document.createElement("button");
      button.innerHTML = `Click Me!`;
      button.addEventListener("click", () => {
        model.set("bytes", new TextEncoder().encode(`Hello World from JavaScript`).buffer);
        model.set("value", "Button Clicked");
        model.save_changes();
      });
      model.on("change:value", () => {
        button.innerHTML = model.get("value");
      });
      el.appendChild(button);
    }
    """
    # Stateful property that can be accessed by JavaScript & Python
    value = traitlets.Unicode('').tag(sync=True)
    bytes = ByteMemoryView().tag(sync=True)

widget = SampleWidget()
widget

In [ ]:
print(bytes(widget.bytes).decode('utf-8'))

In [ ]:
print(widget.value)

In [ ]:
widget.value = "Value from Python"